In [1]:
# 1. collect customer info 2. Survey Customer's Topic Interest 3. Customer Engagement
# sequential task

llm_config={"model": "gpt-3.5-turbo"}

In [2]:
from autogen import ConversableAgent

In [3]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [4]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 05-29 20:04:03] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [5]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 05-29 20:04:03] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 05-29 20:04:03] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [7]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [8]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [9]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


c:\Users\SOUVIK MAJI\Downloads\Autogen\autogen\Lib\site-packages\autogen\agentchat\chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to Onboarding Personal Information Agent):

Souvik from Kolkata

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you for providing that information. To complete your onboarding process, I just need to confirm your name and location. Can you please verify that your name is Souvik and you are located in Kolkata?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

yeah

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Co

In [10]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Souvik",
  "location": "Kolkata"
}


Souvik from Kolkata is interested in reading about football.


Souvik from Kolkata is interested in reading about football. Kolkata has a rich football history, with one of the oldest clubs in Asia, Mohun Bagan AC, founded in 1889, and passionate fans filling up Salt Lake Stadium for epic matches.




In [11]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.0001945, 'gpt-3.5-turbo-0125': {'cost': 0.0001945, 'prompt_tokens': 212, 'completion_tokens': 59, 'total_tokens': 271}}, 'usage_excluding_cached_inference': {'total_cost': 0.0001945, 'gpt-3.5-turbo-0125': {'cost': 0.0001945, 'prompt_tokens': 212, 'completion_tokens': 59, 'total_tokens': 271}}}


{'usage_including_cached_inference': {'total_cost': 5.05e-05, 'gpt-3.5-turbo-0125': {'cost': 5.05e-05, 'prompt_tokens': 68, 'completion_tokens': 11, 'total_tokens': 79}}, 'usage_excluding_cached_inference': {'total_cost': 5.05e-05, 'gpt-3.5-turbo-0125': {'cost': 5.05e-05, 'prompt_tokens': 68, 'completion_tokens': 11, 'total_tokens': 79}}}


{'usage_including_cached_inference': {'total_cost': 0.0003755, 'gpt-3.5-turbo-0125': {'cost': 0.0003755, 'prompt_tokens': 289, 'completion_tokens': 154, 'total_tokens': 443}}, 'usage_excluding_cached_inference': {'total_cost': 0.0003755, 'gpt-3.5-turbo-0125': {'cost': 0.0003755, 'prompt_tokens': 289, 'com